# Задание

Попробуйте настроить параметры градиентного бустинга на этом датасете, и посмотрите, как изменение отдельных параметров влияет на результат.

#### Конечная цель - сказать значение параметров xgb, при которых будет самое хорошее качество на кроссвалидации

In [4]:
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import pandas
import numpy as np
import signal

bioresponce = pandas.read_csv('../seminar01/bioresponse.csv', header=0, sep=',')
bioresponce_target = bioresponce.Activity.values
bioresponce_data = bioresponce.iloc[:, 1:]
print(bioresponce_data[:5])

def signal_handler(signum, frame):
    raise Exception("Timed out!")

def estimate_params(params):
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(30)
    estimator = xgb.XGBClassifier(**params)

    try:
        score = np.mean(cross_val_score(
            estimator, bioresponce_data, bioresponce_target,
            scoring = 'accuracy', cv = 3
        ))
    except Exception:
        score = None
    
    return score

/Users/Timur/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


         D1        D2    D3   D4        D5        D6        D7        D8  \
0  0.000000  0.497009  0.10  0.0  0.132956  0.678031  0.273166  0.585445   
1  0.366667  0.606291  0.05  0.0  0.111209  0.803455  0.106105  0.411754   
2  0.033300  0.480124  0.00  0.0  0.209791  0.610350  0.356453  0.517720   
3  0.000000  0.538825  0.00  0.5  0.196344  0.724230  0.235606  0.288764   
4  0.100000  0.517794  0.00  0.0  0.494734  0.781422  0.154361  0.303809   

         D9       D10  ...    D1767  D1768  D1769  D1770  D1771  D1772  D1773  \
0  0.743663  0.243144  ...        0      0      0      0      0      0      0   
1  0.836582  0.106480  ...        1      1      1      1      0      1      0   
2  0.679051  0.352308  ...        0      0      0      0      0      0      0   
3  0.805110  0.208989  ...        0      0      0      0      0      0      0   
4  0.812646  0.125177  ...        0      0      0      0      0      0      0   

   D1774  D1775  D1776  
0      0      0      0  
1     

In [5]:
print estimate_params({
    'learning_rate': 0.1, 
    'max_depth': 5, 
    'n_estimators': 100, 
    'min_child_weight': 3,
    'seed': 42
})

SyntaxError: invalid syntax (<ipython-input-5-46cb74b27b18>, line 1)

In [6]:
print estimate_params({
    'learning_rate': 0.1, 
    'max_depth': 5, 
    'n_estimators': 100000, 
    'min_child_weight': 3,
    'seed': 42
})

SyntaxError: invalid syntax (<ipython-input-6-06b3d41ffedb>, line 1)

In [12]:
%%time

# My GridSearch
from sklearn.model_selection import GridSearchCV
n_trees = [1] + list(range(5, 55, 5))
parameters = {
    'learning_rate': [0.1, 0.5, 1],
    'max_depth': [1, 2, 3], 
    'n_estimators': n_trees,
    'min_child_weight': [1, 2, 3]
}
"""
    'learning_rate': [0.1, 0.5, 1, 2, 3, 4, 5],
    'max_depth': [1, 2, 3, 4, 5], 
    'n_estimators': n_trees,
    'min_child_weight': [1, 2, 3, 4, 5]
"""
xgb_par = xgb.XGBClassifier()
clf = GridSearchCV(xgb_par, parameters)
clf.fit(bioresponce_data, bioresponce_target)
clf.best_score_
clf.best_params_

CPU times: user 32min 31s, sys: 2min 20s, total: 34min 52s
Wall time: 35min 6s


In [14]:
print(clf.best_score_, clf.best_params_)


0.782458011197 {'learning_rate': 0.5, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 50}


In [33]:
# %%time

# My RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
param_distributions={
    'learning_rate': [0.4, 0.5, 0.8],
    'max_depth': [1, 2], 
    'n_estimators': n_trees,
    'min_child_weight': [0.5, 1, 1.5]
}
res = RandomizedSearchCV(xgb.XGBClassifier(), param_distributions, n_iter=100)
res.fit(bioresponce_data,  bioresponce_target)

"""    
    'learning_rate': [0.4, 0.5, 0.8],
    'max_depth': [1.5, 2, 2.5], 
    'n_estimators': 50,
    'min_child_weight': [0.5, 1, 1.5]
    
"""


print(res.best_params_, res.best_score_)

{'n_estimators': 50, 'min_child_weight': 1, 'max_depth': 2, 'learning_rate': 0.5} 0.782458011197
